In [12]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
from scipy.stats import norm

class Option:
    def __init__(self, option_type, ticker, spot, strike, implied_vol, T, r, q=0, day_counts=252, call_option=True, position=0):
        if option_type not in ["european", "asian"]:
            raise ValueError("Invalid option type. Must be 'european' or 'asian'.")

        self.option_type = option_type
        self.ticker = ticker
        self.spot = spot
        self.strike = strike
        self.implied_vol = implied_vol
        self.T = T
        self.r = r
        self.q = q
        self.day_counts = day_counts
        self.call_option = call_option
        self.position = position

        self.original_price = self.price_option()
        self.current_price = self.original_price

    def price_option(self, num_simulations=1000):
        if self.option_type == "european":
            return self._price_european(num_simulations)
        elif self.option_type == "asian":
            return self._price_asian(num_simulations)

    def calculate_delta(self, num_simulations=1000, epsilon=1e-4):
        original_spot = self.spot
        self.spot += epsilon
        price_up = self.price_option(num_simulations)
        self.spot -= 2 * epsilon
        price_down = self.price_option(num_simulations)
        self.spot = original_spot
        return (price_up - price_down) / (2 * epsilon)

    def calculate_gamma(self, num_simulations=1000, epsilon=1e-4):
        original_spot = self.spot
        self.spot += epsilon
        delta_up = self.calculate_delta(num_simulations, epsilon)
        self.spot -= 2 * epsilon
        delta_down = self.calculate_delta(num_simulations, epsilon)
        self.spot = original_spot
        return (delta_up - delta_down) / (2 * epsilon)

    # def calculate_vega(self, num_simulations=1000, epsilon=1e-4):
    #     original_vol = self.implied_vol
    #     self.implied_vol += epsilon
    #     price_up = self.price_option(num_simulations)
    #     self.implied_vol -= 2 * epsilon
    #     price_down = self.price_option(num_simulations)
    #     self.implied_vol = original_vol
    #     return (price_up - price_down) / (2 * epsilon)

    # def calculate_theta(self, num_simulations=1000, epsilon=1/365):
    #     original_T = self.T
    #     self.T -= epsilon
    #     price_down = self.price_option(num_simulations)
    #     self.T = original_T
    #     return (self.current_price - price_down) / epsilon

    # def calculate_rho(self, num_simulations=1000, epsilon=1e-4):
    #     original_r = self.r
    #     self.r += epsilon
    #     price_up = self.price_option(num_simulations)
    #     self.r -= 2 * epsilon
    #     price_down = self.price_option(num_simulations)
    #     self.r = original_r
    #     return (price_up - price_down) / (2 * epsilon)
    def calculate_vega(self, num_simulations=1000, epsilon=0.01):
        original_vol = self.implied_vol
        self.implied_vol += epsilon
        price_up = self.price_option(num_simulations)
        self.implied_vol -= 2 * epsilon
        price_down = self.price_option(num_simulations)
        self.implied_vol = original_vol
        return (price_up - price_down) / (2 * epsilon) * 0.01

    def calculate_theta(self, num_simulations=1000, epsilon=1/365):
        original_T = self.T
        self.T -= epsilon
        price_down = self.price_option(num_simulations)
        self.T = original_T
        return -(self.current_price - price_down) * (1/365)

    def calculate_rho(self, num_simulations=1000, epsilon=0.0001):
        original_r = self.r
        self.r += epsilon
        price_up = self.price_option(num_simulations)
        self.r -= 2 * epsilon
        price_down = self.price_option(num_simulations)
        self.r = original_r
        return (price_up - price_down) / (2 * epsilon) * 0.01

    def calculate_greeks(self, num_simulations=1000):
        return {
            'delta': self.calculate_delta(num_simulations),
            'gamma': self.calculate_gamma(num_simulations),
            'vega': self.calculate_vega(num_simulations),
            'theta': self.calculate_theta(num_simulations),
            'rho': self.calculate_rho(num_simulations)
        }

    def _price_european(self, num_simulations):
        sim_prices = self._price_modeling(self.spot, self.implied_vol, num_simulations, self.T, self.r, self.q, self.day_counts)
        payoffs = np.maximum(sim_prices.iloc[:, -1] - self.strike, 0) if self.call_option else np.maximum(self.strike - sim_prices.iloc[:, -1], 0)
        return np.mean(payoffs) * np.exp(-self.r * self.T / self.day_counts)

    def _price_modeling(self, S0, iv, N, T, r, q, day_counts):
        t = T / day_counts
        n_steps = int(T)  # Convert T to an integer
        dt = t / n_steps
        prices = {}
        np.random.seed(42)
        for i in range(N):
            W_t = 0
            daily_prices = [S0]
            for j in range(n_steps + 1):
                W_t += np.random.normal(0, 1)
                S_t = S0 * np.exp((r - q - 0.5 * iv**2) * dt + iv * np.sqrt(dt) * W_t)
                daily_prices.append(S_t)
            prices[i] = daily_prices
        return pd.DataFrame.from_dict(prices, orient='index')

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_table = pd.read_html(url)[0]
tickers = sp500_table['Symbol'].tolist()
print("Tickers:", tickers[:10])

end_date = dt.datetime.today()
start_date = end_date - dt.timedelta(days=365)
historical_data = yf.download(tickers, start=start_date, end=end_date, group_by="ticker")
historical_data = historical_data.loc[:, (slice(None), "Close")]
print("Historical Data:", historical_data.head(10))

def generate_portfolio(sp500_data):
    first_date = sp500_data.index[0]
    initial_prices = sp500_data.loc[first_date]
    portfolio_options = []
    for ticker in ['NVDA', 'GOOG', 'MSFT', 'AMZN']:
        spot_price = initial_prices[ticker]['Close']
        print("Spot price:", spot_price)
        portfolio_options.append(Option("european", ticker, spot_price, spot_price * 1.05, 0.3, 63, 0.05, call_option=True, position=100))
    return portfolio_options

backtest = historical_data.iloc[:64]
portfolio = generate_portfolio(historical_data)
print("Portfolio:", [vars(option) for option in portfolio[:10]])

print("Greeks for each option in the portfolio:")
for option in portfolio:
    greeks = option.calculate_greeks()
    print(f"{option.ticker} - Delta: {greeks['delta']:.4f}, Gamma: {greeks['gamma']:.4f}, Vega: {greeks['vega']:.4f}, Theta: {greeks['theta']:.4f}, Rho: {greeks['rho']:.4f}")



Tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


[*********************100%***********************]  503 of 503 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
ERROR:yfinance:['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2023-12-09 04:29:10.998657 -> 2024-12-08 04:29:10.998657)')


Historical Data: Ticker            CNC        PNR        KLAC        CRWD        BAX  \
Price           Close      Close       Close       Close      Close   
Date                                                                  
2023-12-11  74.250000  67.339996  559.210022  246.970001  36.419998   
2023-12-12  76.339996  67.349998  561.049988  248.550003  36.689999   
2023-12-13  76.669998  68.660004  570.700012  252.039993  37.630001   
2023-12-14  75.510002  72.320000  582.419983  252.679993  38.950001   
2023-12-15  74.220001  70.930000  585.130005  260.079987  38.700001   
2023-12-18  74.790001  70.160004  576.729980  259.799988  38.330002   
2023-12-19  74.529999  71.279999  578.109985  258.329987  38.689999   
2023-12-20  72.209999  70.580002  563.299988  253.949997  38.599998   
2023-12-21  73.410004  71.290001  580.450012  257.320007  38.910000   
2023-12-22  73.339996  71.889999  582.650024  255.630005  38.380001   

Ticker            ODFL          GL         RCL         FE  

In [13]:
def calculate_portfolio_over_time(sp500_data, portfolio):
    results = []
    for date in sp500_data.index:
        for option in portfolio[:]:
            option.spot = sp500_data.loc[date, option.ticker]['Close']
            greeks = option.calculate_greeks()
            results.append({
                "Date": date,
                "Ticker": option.ticker,
                "Option Price": option.current_price,
                "Delta": greeks['delta'],
                "Gamma": greeks['gamma'],
                "Vega": greeks['vega'],
                "Theta": greeks['theta'],
                "Rho": greeks['rho']
            })
    return pd.DataFrame(results)

portfolio_df = calculate_portfolio_over_time(backtest, portfolio)
print("Portfolio DataFrame:", portfolio_df.head(10))

Portfolio DataFrame:         Date Ticker  Option Price     Delta         Gamma      Vega     Theta  \
0 2023-12-11   NVDA      2.064436  0.443504  1.110223e-08  0.108588  0.000084   
1 2023-12-11   GOOG      5.963917  0.443504  0.000000e+00  0.313697  0.000242   
2 2023-12-11   MSFT     16.439514  0.443504  8.881784e-08  0.864704  0.000668   
3 2023-12-11   AMZN      6.459361  0.443504 -4.440892e-08  0.339757  0.000263   
4 2023-12-12   NVDA      2.064436  0.505078  1.110223e-08  0.114705  0.001399   
5 2023-12-12   GOOG      5.963917  0.427888 -2.220446e-08  0.307620 -0.000992   
6 2023-12-12   MSFT     16.439514  0.468289  0.000000e+00  0.885534  0.004426   
7 2023-12-12   AMZN      6.459361  0.473424 -2.220446e-08  0.349897  0.002217   
8 2023-12-13   NVDA      2.064436  0.524276 -1.110223e-08  0.116435  0.002003   
9 2023-12-13   GOOG      5.963917  0.433108 -2.220446e-08  0.309609 -0.000617   

        Rho  
0 -0.004340  
1 -0.012539  
2 -0.034564  
3 -0.013581  
4 -0.005431  
5 -

In [15]:
portfolio_df

,Date,Ticker,Option Price,Delta,Gamma,Vega,Theta,Rho
0,2023-12-11,NVDA,2.064436,0.443504,1.110223e-08,0.108588,0.000084,-0.004340
1,2023-12-11,GOOG,5.963917,0.443504,0.000000e+00,0.313697,0.000242,-0.012539
2,2023-12-11,MSFT,16.439514,0.443504,8.881784e-08,0.864704,0.000668,-0.034564
3,2023-12-11,AMZN,6.459361,0.443504,-4.440892e-08,0.339757,0.000263,-0.013581
4,2023-12-12,NVDA,2.064436,0.505078,1.110223e-08,0.114705,0.001399,-0.005431
...,...,...,...,...,...,...,...,...
251,2024-03-12,AMZN,6.459361,0.861588,-8.881784e-08,0.321542,0.054687,-0.060148
252,2024-03-13,NVDA,2.064436,1.000659,0.000000e+00,0.073764,0.111035,-0.102885
253,2024-03-13,GOOG,5.963917,0.547259,-1.332268e-07,0.342081,0.008526,-0.019479
254,2024-03-13,MSFT,16.439514,0.723310,1.776357e-07,0.943303,0.071093,-0.093789
